# Llama 7B 모델 미세 조정하기

## 준비사항 

아래 실습을 원활하게 수행하기 위해 AWS P4d 또는 P4de 인스턴스가 필요합니다.

In [ ]:
%pip install transformers
%pip install peft
%pip install accelerate
%pip install bitsandbytes
%pip install safetensors
%pip install tokenizers
%pip install datasets
%pip install numpy
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121


# %pip install -U transformers==4.39.0
# %pip install -U peft==0.5.0
# %pip install -U accelerate==0.26.0
# #%pip install bitsandbytes #==0.40.2
# #%pip install safetensors==0.3.1"
# #%pip install tokenizers==0.13.3
# %pip install -U datasets==2.17.0
# %pip install --no-cache https://developer.download.nvidia.com/compute/redist/jp/v60dp/pytorch/torch-2.3.0a0+ebedce2.nv24.02-cp310-cp310-linux_aarch64.whl

In [ ]:
import os
import argparse
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    set_seed,
    default_data_collator,
#    BitsAndBytesConfig,
    Trainer,
    TrainingArguments,
)
from datasets import load_dataset
import torch

#import bitsandbytes as bnb
#from huggingface_hub import login, HfFolder

In [ ]:
import argparse
parser = argparse.ArgumentParser()

# 모델 ID와 데이터 세트 경로를 인자에 추가
parser.add_argument(
    "--model_id",
    type=str,
    default="NousResearch/Llama-2-7b-hf", # 게이트 처리 안함
    help="훈련에 사용할 모델 ID.",
)
parser.add_argument(
    "--dataset_path", 
    type=str, 
    default="lm_dataset", 
    help="데이터 세트 경로."
)
# parser.add_argument(
#     "--hf_token", 
#     type=str, 
#     default=HfFolder.get_token(), 
#     help="허깅페이스 사용자 토큰."
# )
# 에포크, 배치 크기, 학습률 및 시드를 위한 훈련 하이퍼파라미터 추가
parser.add_argument(
    "--epochs", 
    type=int, 
    default=1, 
    help="훈련할 에포크 수."
)
parser.add_argument(
    "--per_device_train_batch_size",
    type=int,
    default=1,
    help="훈련에 사용할 배치 크기.",
)
parser.add_argument(
    "--lr", 
    type=float, 
    default=5e-5, 
    help="훈련에 사용할 학습률."
)
parser.add_argument(
    "--seed", 
    type=int, 
    default=42, 
    help="훈련에 사용할 시드."
)
parser.add_argument(
    "--gradient_checkpointing",
    type=bool,
    default=True,
    help="Deepspeed 구성 파일 경로.",
)
parser.add_argument(
    "--bf16",
    type=bool,
    default=True if torch.cuda.get_device_capability()[0] >= 8 else False,
    help="bf16을 사용할지 여부.",
)
# parser.add_argument(
#     "--merge_weights",
#     type=bool,
#     default=True,
#     help="LoRA 가중치를 기본 모델과 병합할지 여부.",
# )
args, _ = parser.parse_known_args()

# if args.hf_token:
#     print(f"허깅페이스 허브에 토큰 {args.hf_token[:10]}로 로그인 중...")
#     login(token=args.hf_token)

In [ ]:
# # https://github.com/artidoro/qlora/blob/main/qlora.py 에서 복사한 코드
# def print_trainable_parameters(model, use_4bit=False):
#     """
#     모델에서 학습 가능한 매개변수의 수를 출력합니다.
#     """
#     trainable_params = 0
#     all_param = 0
#     for _, param in model.named_parameters():
#         num_params = param.numel()
#         # DS Zero 3을 사용하고 가중치가 빈 값으로 초기화된 경우
#         if num_params == 0 and hasattr(param, "ds_numel"):
#             num_params = param.ds_numel
# 
#         all_param += num_params
#         if param.requires_grad:
#             trainable_params += num_params
#     # if use_4bit:
#     #     trainable_params /= 2
#     print(
#         f"전체 파라미터: {all_param:,d} || 학습 가능한 파라미터: {trainable_params:,d} || 학습 가능 비율%: {100 * trainable_params / all_param}"
#     )


# # https://github.com/artidoro/qlora/blob/main/qlora.py 에서 복사한 코드
# def find_all_linear_names(model):
#     lora_module_names = set()
#     # for name, module in model.named_modules():
#     #     if isinstance(module, bnb.nn.Linear4bit):
#     #         names = name.split(".")
#     #         lora_module_names.add(names[0] if len(names) == 1 else names[-1])
#     # 
#     # if "lm_head" in lora_module_names:  # 16비트에 필요
#     #     lora_module_names.remove("lm_head")
#     # return list(lora_module_names)


# def create_peft_model(model, gradient_checkpointing=True, bf16=True):
#     from peft import (
#         get_peft_model,
#         LoraConfig,
#         TaskType,
# #        prepare_model_for_kbit_training,
#     )
#     from peft.tuners.lora import LoraLayer
# 
#     # # 훈련을 위해 int-4 모델 준비
#     # model = prepare_model_for_kbit_training(
#     #     model, use_gradient_checkpointing=gradient_checkpointing
#     # )
#     if gradient_checkpointing:
#         model.gradient_checkpointing_enable()
# 
#     # # LoRA 대상 모듈 가져오기
#     # modules = find_all_linear_names(model)    
#     
#     # 모델의 어텐션 블록만 대상으로 할 경우
#     # modules = ["q_proj", "v_proj"]
# 
#     # 모든 선형 레이어를 대상으로 하는 경우
#     modules = ['q_proj','k_proj','v_proj','o_proj','gate_proj','down_proj','up_proj'] #,'lm_head']
# 
#     print(f"양자화할 모듈 {len(modules)}개를 찾았습니다: {modules}")
# 
#     peft_config = LoraConfig(
#         r=64,
#         lora_alpha=16,
#         target_modules=modules,
#         lora_dropout=0.1,
#         bias="none",
#         task_type=TaskType.CAUSAL_LM,
#     )
# 
#     model = get_peft_model(model, peft_config)
# 
#     # 'norm' 레이어를 float 32로 상향 변환하여 모델 전처리
#     for name, module in model.named_modules():
#         if isinstance(module, LoraLayer):
#             if bf16:
#                 module = module.to(torch.bfloat16)
#         if "norm" in name:
#             module = module.to(torch.float32)
#         if "lm_head" in name or "embed_tokens" in name:
#             if hasattr(module, "weight"):
#                 if bf16 and module.weight.dtype == torch.float32:
#                     module = module.to(torch.bfloat16)
# 
#     model.print_trainable_parameters()
#     return model

## 데이터 세트 불러오기 및 준비

[InstructGPT 논문](https://arxiv.org/abs/2203.02155)에 설명된 여러 행동 카테고리(브레인스토밍, 분류, 폐쇄형 QA, 생성, 정보 추출, 개방형 QA, 요약) 분야에 수천 명의 Databricks 직원들이 생성한 명령어-응답 항목들을 포함한 오픈 소스 데이터 세트 [dolly](https://huggingface.co/datasets/databricks/databricks-dolly-15k)를 사용할 것입니다.

```python
{
  "instruction": "World of Warcraft란 무엇인가?",
  "context": "",
  "response": "World of Warcraft는 대규모 온라인 멀티 플레이어 롤플레잉 게임입니다. 2004년에 Blizzard Entertainment에서 출시했습니다."
}
```

`samsum` 데이터 세트를 불러오려면 허깅페이스 Datasets 라이브러리의 `load_dataset()` 메소드를 사용합니다.

In [ ]:
# 시드 설정하기
set_seed(args.seed)

from datasets import load_dataset
from random import randrange

# 허깅페이스 허브에서 데이터 세트 불러오기
dataset = load_dataset("databricks/databricks-dolly-15k", split="train")
dataset = dataset.select(range(1000))

print(f"데이터 세트 크기: {len(dataset)}")
print(dataset[randrange(len(dataset))])
# 데이터 세트 크기: 15011

모델을 명령어로 조정하려면 구조화된 예제들을 명령어로 표현된 작업들의 모음으로 변환해야 합니다. 샘플을 입력 받아서 지정된 형식의 명령어 문자열을 반환하는 `formatting_function`을 정의합니다.

In [ ]:
def format_dolly(sample):
    instruction = f"### Instruction\n{sample['instruction']}"
    context = f"### Context\n{sample['context']}" if len(sample["context"]) > 0 else None
    response = f"### Answer\n{sample['response']}"
    # 모든 파트를 하나의 문자열로 결합합니다.
    prompt = "\n\n".join([i for i in [instruction, context, response] if i is not None])
    return prompt


포맷팅 함수를 임의의 예제로 테스트 해봅시다.

In [ ]:
from random import randrange

print(format_dolly(dataset[randrange(len(dataset))]))

또한, 샘플을 포맷팅하는 것 외에도 더 효율적인 훈련을 위해 여러 샘플을 하나의 시퀀스로 묶고자 합니다.

In [ ]:
from transformers import AutoTokenizer

#model_id = "meta-llama/Llama-2-13b-hf" # 분할된 가중치, 게이트 처리
model_id = "NousResearch/Llama-2-7b-hf" # 게이트 처리 안함
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

샘플들을 주어진 길이의 시퀀스로 묶고, 이를 토큰화하기 위한 몇 가지 보조 함수를 정의합니다.

In [ ]:
from random import randint
from itertools import chain
from functools import partial


# 각 샘플에 프롬프트를 추가하기 위한 템플릿 데이터 세트
def template_dataset(sample):
    sample["text"] = f"{format_dolly(sample)}{tokenizer.eos_token}"
    return sample


# 각 샘플에 프롬프트 템플릿 적용
dataset = dataset.map(template_dataset, remove_columns=list(dataset.features))
# 임의의 샘플 출력
print(dataset[randint(0, len(dataset))]["text"])

# 다음 배치에서 사용할 수 있도록 배치에서 남은 부분을 저장할 빈 리스트 생성
remainder = {"input_ids": [], "attention_mask": [], "token_type_ids": []}

def chunk(sample, chunk_length=2048):
    # 이전 배치에서 남은 부분을 저장할 전역 변수 remainder 정의
    global remainder
    # 모든 텍스트를 연결하고 이전 배치에서 남은 부분 추가
    concatenated_examples = {k: list(chain(*sample[k])) for k in sample.keys()}
    concatenated_examples = {k: remainder[k] + concatenated_examples[k] for k in concatenated_examples.keys()}
    # 배치의 총 토큰 수 가져오기
    batch_total_length = len(concatenated_examples[list(sample.keys())[0]])

    # 배치에서 최대 청크 수 가져오기
    if batch_total_length >= chunk_length:
        batch_chunk_length = (batch_total_length // chunk_length) * chunk_length

    # 최대 길이의 청크로 분할
    result = {
        k: [t[i : i + chunk_length] for i in range(0, batch_chunk_length, chunk_length)]
        for k, t in concatenated_examples.items()
    }
    # 다음 배치를 위해 전역 변수에 남은 부분 추가
    remainder = {k: concatenated_examples[k][batch_chunk_length:] for k in concatenated_examples.keys()}
    # 레이블 준비
    result["labels"] = result["input_ids"].copy()
    return result


# 데이터 세트를 토큰화하고 청크로 나누기
lm_dataset = dataset.map(
    lambda sample: tokenizer(sample["text"]), batched=True, remove_columns=list(dataset.features)
).map(
    partial(chunk, chunk_length=2048),
    batched=True,
)

# 전체 샘플 수 출력
print(f"전체 샘플 수: {len(lm_dataset)}")

In [ ]:
# 위의 청킹 작업은 행의 수를 줄여줌
print(lm_dataset)

In [ ]:
# bnb 설정을 사용하여 허깅페이스 허브에서 모델 불러오기
# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.bfloat16,
# )

model = AutoModelForCausalLM.from_pretrained(
    args.model_id,
    use_cache=False
    if args.gradient_checkpointing
    else True,  # gradient checkpointing에 필요함
    device_map="auto",
#    quantization_config=bnb_config,
)

# peft 설정 생성
# model = create_peft_model(
#     model, gradient_checkpointing=args.gradient_checkpointing, bf16=args.bf16
# )

In [ ]:
# 훈련 설정 정의하기
output_dir = "./tmp/llama2"
training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=args.per_device_train_batch_size,
    bf16=args.bf16,  # 사용 가능한 경우 BF16 사용
    learning_rate=args.lr,
    num_train_epochs=args.epochs,
    gradient_checkpointing=args.gradient_checkpointing,
    # 로깅 전략
    logging_dir=f"{output_dir}/logs",
    logging_strategy="steps",
    logging_steps=10,
    save_strategy="no",
)

# Trainer 인스턴스 생성
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset,
    data_collator=default_data_collator,
)

# 훈련 시작
trainer.train()

In [ ]:
sagemaker_save_dir="./llama2_dolly"
# if args.merge_weights:
#     # 어댑터 가중치를 기본 모델과 병합하고 저장
#     # int 4 모델 저장
#     trainer.model.save_pretrained(output_dir, safe_serialization=False)
#     # 메모리 정리
#     del model
#     del trainer
#     torch.cuda.empty_cache()

#     from peft import AutoPeftModelForCausalLM

#     # FP16으로 PEFT 모델 불러오기
#     model = AutoPeftModelForCausalLM.from_pretrained(
#         output_dir,
#         low_cpu_mem_usage=True,
#         torch_dtype=torch.bfloat16,
#     )  
#     # LoRA와 기본 모델을 병합하고 저장
#     model = model.merge_and_unload()        
#     model.save_pretrained(
#         sagemaker_save_dir, safe_serialization=True, max_shard_size="2GB"
#     )
# else:

trainer.model.save_pretrained(
    sagemaker_save_dir, safe_serialization=True
)

# 추론을 쉽게 하기 위해 토크나이저 저장
tokenizer = AutoTokenizer.from_pretrained(args.model_id)
tokenizer.save_pretrained(sagemaker_save_dir)